# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [2]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}
for k,v in tuple(obj.items()):
    print(f'{k:^10} = "{v:^3}"')

home_page  = "https://github.com/pypa/sampleproject"
 keywords  = "sample setuptools development"
 license   = "MIT"


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [3]:
import pandas as pd
import re
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj
eq_1 = re.compile(r'[\d]{1}$')
for i in obj:
    print(eq_1.findall(i))

['1']
['4']
['3']


3. Разбейте текст формулировки задачи 2 на слова.

In [4]:
print(re.split(r',| ',"Написать регулярное выражение,которое позволит найти номера групп студентов"))

['Написать', 'регулярное', 'выражение', 'которое', 'позволит', 'найти', 'номера', 'групп', 'студентов']


## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [9]:
import pandas as pd
import numpy as np
import re
recipes = pd.read_csv('recipes_sample.csv')
recipes_r = recipes.sample(5)
size = np.array(recipes_r).dtype.itemsize*2
print(f'|{recipes_r.columns[1]:^{size}}|{recipes_r.columns[2]:^{size}}|\n|{"":-^{size*2+1}}|')
for k,v in zip(recipes_r['id'],recipes_r['minutes']):
    print(f'|{k:^{size}}|{v:^{size}}|')

|       id       |    minutes     |
|---------------------------------|
|     96467      |      320       |
|     63772      |       35       |
|     387644     |       35       |
|     154562     |       10       |
|     236900     |      120       |


In [10]:
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [12]:
import requests
from bs4 import BeautifulSoup
def show_info(file,file2,r_id):
    with open(file2) as f:
        file2 = BeautifulSoup(f, 'xml')
    ids = file2.find_all('id')
    steps = file2.find_all('steps')
    file = file[file['id']==r_id]
    name = file['name'].to_list()[0]
    steps_lst = []
    for k,v in list(zip(ids,steps)):
        if str(r_id) in k:
            for i in v:
                steps_lst.append(i.getText(',',strip=True))
    minutes = file['minutes'].to_list()[0]
    author_id = file['contributor_id'].to_list()[0]
    print(f'"{name}"\n')
    j = 1
    for i in steps_lst[1::2]:
        print(f'{j}. {i}')
        j+=1
    print(f'{"":-<10}\nАвтор: {author_id}')
    print(f'Среднее время приготовления: {minutes} минут')
show_info(recipes,'steps_sample.xml', 170895)

"leeks and parsnips  sauteed or creamed"

1. clean the leeks and discard the dark green portions
2. cut the leeks lengthwise then into one-inch pieces
3. melt the butter in a medium skillet , med
4. heat
5. add the garlic and fry 'til fragrant
6. add leeks and fry until the leeks are tender , about 6-minutes
7. meanwhile , peel and chunk the parsnips into one-inch pieces
8. place in a steaming basket and steam 'til they are as tender as you prefer
9. i like them fork-tender
10. drain parsnips and add to the skillet with the leeks
11. add salt and pepper
12. gently sautee together for 5-minutes
13. at this point you can serve it , or continue on and cream it:
14. in a jar with a screw top , add the half-n-half and arrowroot
15. shake 'til blended
16. turn heat to low under the leeks and parsnips
17. pour in the arrowroot mixture , stirring gently as you pour
18. if too thick , gradually add the water
19. let simmer for a couple of minutes
20. taste to adjust seasoning , probably an addi

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [13]:
with open('steps_sample.xml') as f:
        file2 = BeautifulSoup(f, 'xml')
stps_lst = []
ids = file2.find_all('id')
steps = file2.find_all('steps')
for k,v in list(zip(ids,steps)):
    if str(25082) in k:
         for i in v:
            stps_lst.append(i.getText(',',strip=True))
eq = re.compile(r'[\d]{1,}[\s]hours|[\d]{1,}[\s]hour|[\d]{1,}[\s]minutes|[\d]{1,}[\s]minute')
for i in stps_lst:
    if len(eq.findall(i)) >= 1:
        print(eq.findall(i))

['20 minutes']
['10 minutes']
['2 hours']
['10 minutes']
['20 minutes', '30 minutes']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [14]:
eq1 = re.compile(r'^this[\s\w\d_]{1,}[,][ ]but|^this[\s\w\d_]{1,}[,]but')
lst1 = []
for i in recipes['description'].values.tolist():
    if type(i) == str:
        if len(eq1.findall(i)) >=1:
            lst1.append(i)
print(f'Количество рецептов, удовлетворяющих выражению = {len(lst1)}\nПримеры: {lst1[:3]}')

Количество рецептов, удовлетворяющих выражению = 134
Примеры: ["this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.", "this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.", 'this is kind of similar to some of the other versions out there, but it is the best and easiest i have found']


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [15]:
eq2 = re.compile(r'[\s]/[\s]')
stps_lst2 = []
for k,v in list(zip(ids,steps)):
    if str(72367) in k:
         for i in v:
            stps_lst2.append(i.getText(',',strip=True))
for i in stps_lst2[1::2]:
    print(eq2.sub('/',i))

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [37]:
import nltk
# nltk.download()
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk import pos_tag

In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zerka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\zerka\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [20]:
lst = []
for i in file2.find_all("recipe"):
    for j in i.find_all("step"):
        lst_add = [word.lower() for word in word_tokenize(j.get_text()) if word.isalpha()]
        lst+=lst_add
k = list(set(lst))
len(k)

14926

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [28]:
recipes['sent_tokenize'] = recipes.description.fillna("").apply(nltk.sent_tokenize)
five_recipes = recipes.sort_values(by="sent_tokenize", key= lambda x: x.str.len(),ascending=False).iloc[:5]
five_recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,sent_tokenize
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN,[this wonderful icing is used for icing cakes ...
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0,[a translucent golden-brown crust allows the g...
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0,[this is one of the best soups i've ever made ...
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN,[the first time i made this cake i grated a mi...
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN,[i wrote this because there are an astounding ...


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [57]:
def pos_tagg(sntc):
    tagged_words = pos_tag(word_tokenize(sntc))
    pos_tags,words = [tag for word, tag in tagged_words],[word for word, tag in tagged_words]

    for tag, word in zip(pos_tags,words):
        print(f'{tag:^{len(word)+2}}', end=' ')
    print()
    for tag, word in zip(pos_tags,words):
        print(f'{word:^{len(word)+2}}', end=' ')
    print('\n')
pos_tagg('I  omitted the raspberries and added strawberries instead')
pos_tagg(recipes[recipes['id'] == 241106].iloc[0]['name'])

PRP    VBD     DT        NNS       CC     VBD        NNS          RB     
 I   omitted   the   raspberries   and   added   strawberries   instead  

    JJ       NNS      IN       NNS      VBP      JJ     CC     JJ      NNS    
 eggplant   steaks   with   chickpeas   feta   cheese   and   black   olives  

